In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from urllib.parse import urlparse

In [3]:
url_base='https://www.vhcy.gov.tw'
url_work_table='/PageView/RowView?WebMenuID=1c791b28-2968-49c9-8d5a-32dceca8ad1b'
url=url_base+url_work_table
g=requests.get(url)
soup=BeautifulSoup(g.content, 'html.parser')

# Get page link
In this website the work table has some page, so the function should find page link for next step.

In [14]:
pages_link=[]
pages_link.append(url)
for o in soup.find("div",class_="pager").find_all("a"):
    if o.text == "下一頁" or o.text == "»":
        pass
    else:
        link = url_base+o.get('href')
        pages_link.append(link)
        print(o.text,link)

2 https://www.vhcy.gov.tw/PageView/RowView?WebMenuID=1c791b28-2968-49c9-8d5a-32dceca8ad1b&page=2
3 https://www.vhcy.gov.tw/PageView/RowView?WebMenuID=1c791b28-2968-49c9-8d5a-32dceca8ad1b&page=3
4 https://www.vhcy.gov.tw/PageView/RowView?WebMenuID=1c791b28-2968-49c9-8d5a-32dceca8ad1b&page=4
5 https://www.vhcy.gov.tw/PageView/RowView?WebMenuID=1c791b28-2968-49c9-8d5a-32dceca8ad1b&page=5
6 https://www.vhcy.gov.tw/PageView/RowView?WebMenuID=1c791b28-2968-49c9-8d5a-32dceca8ad1b&page=6


# Get work table

The `get_work_page` function is used for getting each work page

In [20]:
work_page = soup.find("tbody")

In [23]:
def get_work_page(page_url,page=None,page_link_part='?page='):
    global url
    if page != None:
        page_url = page_url+page_link_part+str(page)
    if url == None:
        page_url = url
        print(page_url)
    g=requests.get(page_url)
    soup=BeautifulSoup(g.content, 'html.parser')
    return soup

In [178]:
def get_work_table(table_,work_table):
    for p_i, p_item in enumerate(table_.find_all("tr")):
        if p_item.find('a'):
            p_item_a = p_item.find('a')
            title = p_item_a.text
            if "錄取公告" in title:
                break
            try:
                new_title = re.search("\B[科,短,社,契,牙,秘,醫,部,室](.*)[員,理,工,師,生]",title).group(0)
                origination = re.search("((.*)[組,部,科,室,心])",title).group(1)
            except AttributeError:
                print("ERROR:",title)
            
            link_s = url_base+'/'+p_item_a.get('href')
            
            try:
                dead_line = get_work_deadtime(link_s)
            except:
                dead_line = "null(check webpage)"
            print(p_i-1,new_title,link_s,dead_line, origination)
            work_table.append([p_i-1, title, dead_line, link_s ])
    return work_table

In [179]:
def get_work_detail(link):
    return get_work_page(link).find("div",class_="newContent").text.replace("\r","").replace("\t","").split("\n")

In [180]:
work_table=[]
work_table = get_work_table(work_page,work_table)
for p_i, p_item in enumerate(pages_link):
    table_ = get_work_page(p_item)

    work_table = get_work_table(table_,work_table)

-1 室專案短期助理 https://www.vhcy.gov.tw//PageView/RowViewDetail?WebRowsID=45afd34a-e70e-4988-be1c-898b62d4a15a&UnitID=fbb1c734-f7d0-41c4-93d8-261f32f1db04&CompanyID=E8E0488E-54A0-44BF-B10C-D029C423F6E7 null(check webpage) 秘書室
0 科助理 https://www.vhcy.gov.tw//PageView/RowViewDetail?WebRowsID=1f64c98e-7dc1-474e-818e-f886fdaff6cb&UnitID=fbb1c734-f7d0-41c4-93d8-261f32f1db04&CompanyID=E8E0488E-54A0-44BF-B10C-D029C423F6E7 null(check webpage) 牙科
1 室契約健保申報員 https://www.vhcy.gov.tw//PageView/RowViewDetail?WebRowsID=83476288-ad39-4918-90e6-5dfb43d77b42&UnitID=fbb1c734-f7d0-41c4-93d8-261f32f1db04&CompanyID=E8E0488E-54A0-44BF-B10C-D029C423F6E7 null(check webpage)  醫務企管室
2 室契約行政助理 https://www.vhcy.gov.tw//PageView/RowViewDetail?WebRowsID=a9ca6f7e-d6d6-4ecb-850f-0c14eb49dfc7&UnitID=fbb1c734-f7d0-41c4-93d8-261f32f1db04&CompanyID=E8E0488E-54A0-44BF-B10C-D029C423F6E7 null(check webpage) 醫務企管室
3 室專案社工 https://www.vhcy.gov.tw//PageView/RowViewDetail?WebRowsID=6891bd3b-a04a-4a9f-b2df-3e6065e4b6d1&UnitID=fbb1c734

UnboundLocalError: local variable 'origination' referenced before assignment

In [90]:
work_page.find_all("tr")[0]

<tr>
<td data-th="標題" style="text-align:left;">
<span style="width:100%;white-space:normal;word-wrap:break-word;word-break:break-all;">
<a href="/PageView/RowViewDetail?WebRowsID=45afd34a-e70e-4988-be1c-898b62d4a15a&amp;UnitID=fbb1c734-f7d0-41c4-93d8-261f32f1db04&amp;CompanyID=E8E0488E-54A0-44BF-B10C-D029C423F6E7" title="秘書室專案短期助理1名徵才公告">秘書室專案短期助理1名徵才公告</a>
</span>
</td>
<td data-th="更新日期">2022/5/18</td>
</tr>

In [94]:
g =get_work_detail('https://www.vhcy.gov.tw/PageView/RowViewDetail?WebRowsID=83476288-ad39-4918-90e6-5dfb43d77b42&UnitID=fbb1c734-f7d0-41c4-93d8-261f32f1db04&CompanyID=E8E0488E-54A0-44BF-B10C-D029C423F6E7')

In [105]:
 [gg for gg in g if "職稱" in gg][0].split("：")[1]

' 契約健保申報員'

In [107]:
[gg for gg in g if "職系" in gg or "用人單位" in gg][0].split("：")[1]

' 醫務企管室-醫療費用組'

In [184]:
re.search("\B[灣,科,短,社,契,牙,秘,醫,部,室,師](.*)[員,理,工,師,生]","臺中榮民總醫院嘉義分院暨灣橋分院師(三)級醫師32名外補職缺公告").group(1)

'院嘉義分院暨灣橋分院師(三)級醫'

In [108]:
re.search("\B[專案,契]((.*)[員,師,助理])","醫務企管室契約行政助理1名徵才公告").group(1)

'約行政助理'

In [88]:
g

'職系： 醫務企管室-醫療費用組職稱： 契約健保申報員名額：2名（得依需要列候補2名，有效期間自甄選結果確定之翌日起4個月內）上網期間：111年04月27日至111年05月27日資格條件 需具備下列條件：1.學歷：國內、外高中以上學校-護理、醫管、資訊或商學與管理相關科系畢業或具醫院行政工作相關經驗者。2.證照：具健康保險技術員證照者尤佳。3.技能：熟Office電腦操作及統計分析能力，並有良好溝通協調能力者。4.人格特質：樂觀進取、認真負責、言行端正，具高度工作熱忱、態度積極配合度佳，並有良好溝通及學習力。工作項目(一)健保申報、抽審、申覆、爭議相關業務。(二)協助推動執行本院健保政策相關業務。(三)統計分析報表。(四)其他臨時交辦事項。薪資範圍 26,510 ~32,150元(不含獎勵金)聯絡方式： 1.報名人員請於111年05月15日前報名。2.請檢附履歷表（橫式一般、內貼二吋半身正面照片並註明連絡電話）、最高學歷畢業證書、相關證照、證書(影本)（A4規格）。3.不符合前開資格條件或經審查不合業務需要者，恕不通知及退件。4.以郵寄報名（以郵戳為?，逾期不受理）：嘉義市世賢路2段600號或電子郵件報名：E-mail：tsing@vghtc.gov.tw，醫企室醫療費用組戴小姐收，分機：6026。5.考試方式:筆試及口試。備註：「依國軍退除役官兵輔導條例第六條規定，條件相等而為退除役官兵者，優先錄用」'